In [2]:
import pickle

with open('dataframe.pkl', 'rb') as file:
    df = pickle.load(file)

In [3]:
from sklearn.model_selection import train_test_split

# Séparez les données en fonction de la variable 'Flagimpaye'
df_pas_utilise, df_reduit = train_test_split(df, test_size=0.01, stratify=df['FlagImpaye'], random_state=42)

# Affichez la taille du sous-DataFrame de test pour vérifier
print("Taille du sous-DataFrame de test :", len(df_reduit))

Taille du sous-DataFrame de test : 46468


In [4]:
df_reduit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46468 entries, 4207882 to 873112
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   ZIBZIN                    46468 non-null  object        
 1   IDAvisAutorisationCheque  46468 non-null  object        
 2   FlagImpaye                46468 non-null  category      
 3   Montant                   46468 non-null  float64       
 4   DateTransaction           46468 non-null  datetime64[ns]
 5   CodeDecision              46468 non-null  object        
 6   VerifianceCPT1            46468 non-null  int64         
 7   VerifianceCPT2            46468 non-null  int64         
 8   VerifianceCPT3            46468 non-null  int64         
 9   D2CB                      46468 non-null  int64         
 10  ScoringFP1                46468 non-null  float64       
 11  ScoringFP2                46468 non-null  float64       
 12  ScoringFP3 

### On définit la partie Train et Test

In [5]:
# Définition des dates limites pour les ensembles d'apprentissage et de test
train_inf = '2017-02-01'
train_sup = '2017-08-31'
test_inf = '2017-09-01'
test_sup = '2017-11-30'

In [6]:
train = df_reduit.loc[(df_reduit['DateTransaction'] >= train_inf) & (df_reduit['DateTransaction'] <= train_sup)]
X_train = train.drop(columns=['FlagImpaye','CodeDecision','DateTransaction','ZIBZIN' , 'IDAvisAutorisationCheque'])
y_train = train['FlagImpaye']

In [7]:
test = df_reduit.loc[(df_reduit['DateTransaction'] >= test_inf) & (df_reduit['DateTransaction'] <= test_sup)]
X_test = test.drop(columns=['FlagImpaye','CodeDecision','DateTransaction','ZIBZIN' , 'IDAvisAutorisationCheque'])
y_test = test['FlagImpaye']

## Test de chaque méthode d'échantillonage par modeles

In [8]:
from sklearn import metrics
from sklearn.metrics import f1_score

In [9]:
!pip install 'imblearn'
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN, BorderlineSMOTE

In [10]:
!pip install xgboost

On fait tourner DecisionTreeClassifier, RandomForestClassifier, GradientBoostingClassifier,KNeighborsClassifier, SVC

In [11]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import RandomOverSampler, SMOTE, BorderlineSMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from xgboost import XGBClassifier

# Définir les stratégies d'échantillonnage
sampling_strategies = [0.01, 0.05, 0.1]  # Ajoutez les valeurs que vous souhaitez tester
samplers = [RandomOverSampler, SMOTE, BorderlineSMOTE, ADASYN, RandomUnderSampler, NearMiss]

# Définir les modèles à tester
models = [
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    GradientBoostingClassifier(),
    KNeighborsClassifier(),
    SVC(),
]

# Initialiser un DataFrame pour stocker les résultats
results_df = pd.DataFrame(columns=['Model', 'Sampler', 'Sampling Strategy', 'F1 Score'])

# Parcourir chaque modèle
for model in models:
    # Parcourir chaque échantillonneur
    for sampler in samplers:
        # Parcourir chaque valeur de sampling_strategy
        for strategy in sampling_strategies:
            # Appliquer l'échantillonnage
            X_resampled, y_resampled = sampler(sampling_strategy=strategy).fit_resample(X_train, y_train)

            # Initialiser et entraîner le modèle
            model.fit(X_resampled, y_resampled)

            # Faire des prédictions sur l'ensemble de test
            y_pred = model.predict(X_test)

            # Évaluer les performances du modèle
            f1 = f1_score(y_test, y_pred)

            # Ajouter les résultats au DataFrame
            results_df = results_df.append({
                'Model': model.__class__.__name__,
                'Sampler': sampler.__name__,
                'Sampling Strategy': strategy,
                'F1 Score': f1
            }, ignore_index=True)

# Afficher le tableau récapitulatif
print(results_df)


/var/folders/0b/1f6rtdh943b_w3yh5c9675cw0000gn/T/ipykernel_32453/1653289211.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
/var/folders/0b/1f6rtdh943b_w3yh5c9675cw0000gn/T/ipykernel_32453/1653289211.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
/var/folders/0b/1f6rtdh943b_w3yh5c9675cw0000gn/T/ipykernel_32453/1653289211.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
/var/folders/0b/1f6rtdh943b_w3yh5c9675cw0000gn/T/ipykernel_32453/1653289211.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.appen

                     Model             Sampler Sampling Strategy  F1 Score
0   DecisionTreeClassifier   RandomOverSampler              0.01  0.027027
1   DecisionTreeClassifier   RandomOverSampler              0.05  0.052632
2   DecisionTreeClassifier   RandomOverSampler               0.1  0.029412
3   DecisionTreeClassifier               SMOTE              0.01  0.052356
4   DecisionTreeClassifier               SMOTE              0.05   0.04142
..                     ...                 ...               ...       ...
85                     SVC  RandomUnderSampler              0.05       0.0
86                     SVC  RandomUnderSampler               0.1       0.0
87                     SVC            NearMiss              0.01       0.0
88                     SVC            NearMiss              0.05       0.0
89                     SVC            NearMiss               0.1       0.0

[90 rows x 4 columns]


/var/folders/0b/1f6rtdh943b_w3yh5c9675cw0000gn/T/ipykernel_32453/1653289211.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


On fait tourner à part AdaBoostClassifier pcq pas sûr que ça marche

In [12]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import RandomOverSampler, SMOTE, BorderlineSMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from xgboost import XGBClassifier

# Définir les stratégies d'échantillonnage
sampling_strategies = [0.01, 0.05, 0.1]  # Ajoutez les valeurs que vous souhaitez tester
samplers = [RandomOverSampler, SMOTE, BorderlineSMOTE, ADASYN, RandomUnderSampler, NearMiss]

# Définir les modèles à tester
models = [
    AdaBoostClassifier()
]

# Initialiser un DataFrame pour stocker les résultats
results_df2 = pd.DataFrame(columns=['Model', 'Sampler', 'Sampling Strategy', 'F1 Score'])

# Parcourir chaque modèle
for model in models:
    # Parcourir chaque échantillonneur
    for sampler in samplers:
        # Parcourir chaque valeur de sampling_strategy
        for strategy in sampling_strategies:
            # Appliquer l'échantillonnage
            X_resampled, y_resampled = sampler(sampling_strategy=strategy).fit_resample(X_train, y_train)

            # Initialiser et entraîner le modèle
            model.fit(X_resampled, y_resampled)

            # Faire des prédictions sur l'ensemble de test
            y_pred = model.predict(X_test)

            # Évaluer les performances du modèle
            f1 = f1_score(y_test, y_pred)

            # Ajouter les résultats au DataFrame
            results_df2 = results_df2.append({
                'Model': model.__class__.__name__,
                'Sampler': sampler.__name__,
                'Sampling Strategy': strategy,
                'F1 Score': f1
            }, ignore_index=True)

# Afficher le tableau récapitulatif
print(results_df2)


/var/folders/0b/1f6rtdh943b_w3yh5c9675cw0000gn/T/ipykernel_32453/675476911.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df2 = results_df2.append({
/var/folders/0b/1f6rtdh943b_w3yh5c9675cw0000gn/T/ipykernel_32453/675476911.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df2 = results_df2.append({
/var/folders/0b/1f6rtdh943b_w3yh5c9675cw0000gn/T/ipykernel_32453/675476911.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df2 = results_df2.append({
/var/folders/0b/1f6rtdh943b_w3yh5c9675cw0000gn/T/ipykernel_32453/675476911.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df2 = results_df2.a

                 Model             Sampler Sampling Strategy  F1 Score
0   AdaBoostClassifier   RandomOverSampler              0.01       0.0
1   AdaBoostClassifier   RandomOverSampler              0.05  0.065217
2   AdaBoostClassifier   RandomOverSampler               0.1  0.068966
3   AdaBoostClassifier               SMOTE              0.01  0.027027
4   AdaBoostClassifier               SMOTE              0.05  0.042553
5   AdaBoostClassifier               SMOTE               0.1  0.058824
6   AdaBoostClassifier     BorderlineSMOTE              0.01       0.0
7   AdaBoostClassifier     BorderlineSMOTE              0.05  0.066667
8   AdaBoostClassifier     BorderlineSMOTE               0.1  0.042781
9   AdaBoostClassifier              ADASYN              0.01       0.0
10  AdaBoostClassifier              ADASYN              0.05  0.067227
11  AdaBoostClassifier              ADASYN               0.1  0.076923
12  AdaBoostClassifier  RandomUnderSampler              0.01       0.0
13  Ad

/var/folders/0b/1f6rtdh943b_w3yh5c9675cw0000gn/T/ipykernel_32453/675476911.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df2 = results_df2.append({


On fait tourner à part XGBClassifier pcq pas sûr que ça marche

In [13]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import RandomOverSampler, SMOTE, BorderlineSMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from xgboost import XGBClassifier

# Définir les stratégies d'échantillonnage
sampling_strategies = [0.01, 0.05, 0.1]  # Ajoutez les valeurs que vous souhaitez tester
samplers = [RandomOverSampler, SMOTE, BorderlineSMOTE, ADASYN, RandomUnderSampler, NearMiss]

# Définir les modèles à tester
models = [
    XGBClassifier()
]

# Initialiser un DataFrame pour stocker les résultats
results_df3 = pd.DataFrame(columns=['Model', 'Sampler', 'Sampling Strategy', 'F1 Score'])

# Parcourir chaque modèle
for model in models:
    # Parcourir chaque échantillonneur
    for sampler in samplers:
        # Parcourir chaque valeur de sampling_strategy
        for strategy in sampling_strategies:
            # Appliquer l'échantillonnage
            X_resampled, y_resampled = sampler(sampling_strategy=strategy).fit_resample(X_train, y_train)

            # Initialiser et entraîner le modèle
            model.fit(X_resampled, y_resampled)

            # Faire des prédictions sur l'ensemble de test
            y_pred = model.predict(X_test)

            # Évaluer les performances du modèle
            f1 = f1_score(y_test, y_pred)

            # Ajouter les résultats au DataFrame
            results_df3 = results_df3.append({
                'Model': model.__class__.__name__,
                'Sampler': sampler.__name__,
                'Sampling Strategy': strategy,
                'F1 Score': f1
            }, ignore_index=True)

# Afficher le tableau récapitulatif
print(results_df3)


/var/folders/0b/1f6rtdh943b_w3yh5c9675cw0000gn/T/ipykernel_32453/432830665.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df3 = results_df3.append({
/var/folders/0b/1f6rtdh943b_w3yh5c9675cw0000gn/T/ipykernel_32453/432830665.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df3 = results_df3.append({
/var/folders/0b/1f6rtdh943b_w3yh5c9675cw0000gn/T/ipykernel_32453/432830665.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df3 = results_df3.append({
/var/folders/0b/1f6rtdh943b_w3yh5c9675cw0000gn/T/ipykernel_32453/432830665.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df3 = results_df3.a

            Model             Sampler Sampling Strategy  F1 Score
0   XGBClassifier   RandomOverSampler              0.01  0.026316
1   XGBClassifier   RandomOverSampler              0.05   0.02439
2   XGBClassifier   RandomOverSampler               0.1  0.084211
3   XGBClassifier               SMOTE              0.01       0.0
4   XGBClassifier               SMOTE              0.05  0.104348
5   XGBClassifier               SMOTE               0.1  0.084034
6   XGBClassifier     BorderlineSMOTE              0.01  0.066667
7   XGBClassifier     BorderlineSMOTE              0.05  0.060606
8   XGBClassifier     BorderlineSMOTE               0.1  0.040404
9   XGBClassifier              ADASYN              0.01  0.072289
10  XGBClassifier              ADASYN              0.05  0.070423
11  XGBClassifier              ADASYN               0.1  0.031008
12  XGBClassifier  RandomUnderSampler              0.01  0.046512
13  XGBClassifier  RandomUnderSampler              0.05  0.077295
14  XGBCla

/var/folders/0b/1f6rtdh943b_w3yh5c9675cw0000gn/T/ipykernel_32453/432830665.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df3 = results_df3.append({


On réunit les résultats dans un seul tableau recapitulatif

In [14]:
concatenated_results_df = pd.concat([results_df, results_df2, results_df3], ignore_index=True)

In [20]:
concatenated_results_df

,Model,Sampler,Sampling Strategy,F1 Score
0,DecisionTreeClassifier,RandomOverSampler,0.01,0.027027
1,DecisionTreeClassifier,RandomOverSampler,0.05,0.052632
2,DecisionTreeClassifier,RandomOverSampler,0.1,0.029412
3,DecisionTreeClassifier,SMOTE,0.01,0.052356
4,DecisionTreeClassifier,SMOTE,0.05,0.04142
...,...,...,...,...
121,XGBClassifier,RandomUnderSampler,0.05,0.077295
122,XGBClassifier,RandomUnderSampler,0.1,0.052045
123,XGBClassifier,NearMiss,0.01,0.080808
124,XGBClassifier,NearMiss,0.05,0.055147


In [22]:
concatenated_results_df.describe(include='all')

,Model,Sampler,Sampling Strategy,F1 Score
count,126,126,126.00,126.0
unique,7,6,3.00,78.0
top,DecisionTreeClassifier,RandomOverSampler,0.01,0.0
freq,18,21,42.00,37.0


In [24]:
# Grouper les résultats par modèle
grouped_results = concatenated_results_df.groupby('Model')

# Initialiser un DataFrame pour stocker les meilleurs résultats par modèle
best_results_df = pd.DataFrame(columns=['Model', 'Sampler', 'Sampling Strategy', 'F1 Score'])

# Parcourir chaque groupe
for model, group_df in grouped_results:
    # Trouver la ligne avec le plus grand F1 score pour chaque modèle
    best_result = group_df.sort_values(by='F1 Score', ascending=False).iloc[0]
    
    # Ajouter le meilleur résultat au DataFrame final
    best_results_df = best_results_df.append(best_result, ignore_index=True)

# Afficher le tableau récapitulatif des meilleurs résultats pour chaque modèle
print(best_results_df)


                        Model             Sampler Sampling Strategy  F1 Score
0          AdaBoostClassifier  RandomUnderSampler               0.1  0.110345
1      DecisionTreeClassifier     BorderlineSMOTE               0.1  0.075269
2  GradientBoostingClassifier  RandomUnderSampler               0.1  0.124294
3        KNeighborsClassifier               SMOTE               0.1  0.039604
4      RandomForestClassifier  RandomUnderSampler              0.05  0.095238
5                         SVC   RandomOverSampler              0.01       0.0
6               XGBClassifier               SMOTE              0.05  0.104348


/var/folders/0b/1f6rtdh943b_w3yh5c9675cw0000gn/T/ipykernel_32453/3836614240.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  best_results_df = best_results_df.append(best_result, ignore_index=True)
/var/folders/0b/1f6rtdh943b_w3yh5c9675cw0000gn/T/ipykernel_32453/3836614240.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  best_results_df = best_results_df.append(best_result, ignore_index=True)
/var/folders/0b/1f6rtdh943b_w3yh5c9675cw0000gn/T/ipykernel_32453/3836614240.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  best_results_df = best_results_df.append(best_result, ignore_index=True)
/var/folders/0b/1f6rtdh943b_w3yh5c9675cw0000gn/T/ipykernel_32453/3836614240.py:13: FutureWarning: The frame.append method is de

In [25]:
best_results_df

,Model,Sampler,Sampling Strategy,F1 Score
0,AdaBoostClassifier,RandomUnderSampler,0.1,0.110345
1,DecisionTreeClassifier,BorderlineSMOTE,0.1,0.075269
2,GradientBoostingClassifier,RandomUnderSampler,0.1,0.124294
3,KNeighborsClassifier,SMOTE,0.1,0.039604
4,RandomForestClassifier,RandomUnderSampler,0.05,0.095238
5,SVC,RandomOverSampler,0.01,0.0
6,XGBClassifier,SMOTE,0.05,0.104348


## Maintenant qu'on a une vue d'ensemble claire sur les méthodes de re-échantillonage qui marchent le mieux en fonction des modèles, on va reprendre ces combinaisons pour réaliser une optimisation cette fois-ci des hyper-paramètres des modèles à l'aide d'un greedsearch.

### Model: GradientBoostingClassifier - Sampler: RandomUnderSampler - Sampling Strategy: 0.1

In [51]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline

# Sous-échantillonnage
rus = RandomUnderSampler(sampling_strategy=0.1)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

# Modèle de l'arbre de décision
gb_classifier = GradientBoostingClassifier()

# Pipeline
pipeline = Pipeline([
    ('classifier', gb_classifier)
])

# Créer un objet TimeSeriesSplit pour la cross-validation
tscv = TimeSeriesSplit(n_splits = 20)

# Paramètres pour la recherche sur la grille
param_grid = {
    'classifier__n_estimators': [50, 100],
    'classifier__max_depth': [5, 10, 15],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [2, 4]
}

# Recherche sur la grille
grid_search = GridSearchCV(pipeline, param_grid, scoring='f1', cv=tscv)
grid_search.fit(X_train_rus, y_train_rus)

# Afficher les meilleurs paramètres et le meilleur score F1
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score F1:", grid_search.best_score_)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
684 fits failed out of a total of 720.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
684 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py", line 497, in fit
    y = self._validate_y(y, sample_weight)
  File "/opt/anacon

Meilleurs paramètres: {'classifier__max_depth': 5, 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 50}
Meilleur score F1: nan


In [44]:
import numpy as np

# Compter le nombre d'éléments uniques dans y_train_rus
unique_classes, counts = np.unique(y_train_rus, return_counts=True)

# Afficher le nombre d'éléments uniques et leurs occurrences
for cls, count in zip(unique_classes, counts):
    print(f"Classe {cls}: {count} occurrences")


Classe 0: 2290 occurrences
Classe 1: 229 occurrences


In [53]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline

# Sous-échantillonnage
rus = RandomUnderSampler(sampling_strategy=0.1)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

# Modèle de l'AdaBoostClassifier
adab_classifier = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())  # Utilise un arbre de décision faible par défaut

# Pipeline
pipeline = Pipeline([
    ('classifier', adab_classifier)
])

# Créer un objet TimeSeriesSplit pour la cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Paramètres pour la recherche sur la grille
param_grid = {
    'classifier__n_estimators': [50, 100],
    'classifier__base_estimator__max_depth': [1, 2, 3],  # Ajustez ici pour le paramètre de l'arbre faible
    'classifier__learning_rate': [0.1, 0.5, 1.0],
}

# Recherche sur la grille
grid_search = GridSearchCV(pipeline, param_grid, scoring='f1', cv=tscv)
grid_search.fit(X_train_rus, y_train_rus)

# Afficher les meilleurs paramètres et le meilleur score F1
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score F1:", grid_search.best_score_)


/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/anaconda3/lib/python3.9/site-packages/skl

Meilleurs paramètres: {'classifier__base_estimator__max_depth': 1, 'classifier__learning_rate': 0.1, 'classifier__n_estimators': 50}
Meilleur score F1: 0.0
